# Próba stworzenia nowego modelu


In [68]:
import torch
from torch_geometric.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Wczytaj zbiór uczący
train_dataset = torch.load('train_dataset.pt')
# wczytaj zbiór testowy
test_dataset = torch.load('test_dataset.pt')

print(len(train_dataset))
print(len(test_dataset))

# Definiuj DataLoader dla zbiorów uczącego i testowego
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

184
46


/home/franek/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


# Model F01.12a-słaby

In [243]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv, EdgeConv

class CustomEdgeConv(nn.Module):
    def __init__(self, num_node_features, hidden_dim):
        super(CustomEdgeConv, self).__init__()
        self.layers = nn.ModuleList([
            nn.Linear(1, 32),  # Adjust the input size for the first linear layer
            nn.ReLU(),
            nn.Linear(32, hidden_dim)
        ])

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x)
        return x

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        #self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Edge Convolutional Layer
        self.conv2 = CustomEdgeConv(num_node_features, hidden_dim+2)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 4, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        #x = self.conv1(x, edge_index, edge_weight=edge_weight)
        # Apply Edge Convolution
        x = self.conv2(x, edge_index)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# Model F01.12b

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
    

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GCNConv(num_node_features, hidden_dim)


        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)


        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# szkolenie

In [244]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout).to(device)
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        data.to(device)
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)
# Open the file in 'a' mode (append mode)
with open("file.txt", "a") as file:
    # Append the text to the file
    file.write("\n" + "text_to_append")  # Add a new line before appending

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/tmp/ipykernel_11771/1014723709.py:55: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [09:04<00:00, 54.47s/trial, best loss: 96876.62628173828]
Najlepsze hiperparametry: {'dropout': 0.1516100937368105, 'hidden_dim': 128.0}


In [245]:
import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

#optimal_hidden_dim = 192
#optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout).to(device)



# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 100

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0].to(device)

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Epoch: 0


/tmp/ipykernel_11771/1014723709.py:55: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)


Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Przewidywana wartość: 651.4077758789062
Rzeczywista wartość: 874.5499877929688


In [62]:
sample_data = test_dataset[30]

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data.to(device))[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Przewidywana wartość: 824.8858032226562
Rzeczywista wartość: 860.9299926757812


In [64]:
sample_data = test_dataset[9]

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data.to(device))[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Przewidywana wartość: 283.0379943847656
Rzeczywista wartość: 286.3299865722656


In [65]:
total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data.to(device))[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")


Przykład 1:
Przewidywana wartość: 828.5482177734375
Rzeczywista wartość: 874.5499877929688
Procentowa jakość regresji: 94.74%

Przykład 2:
Przewidywana wartość: 571.54931640625
Rzeczywista wartość: 569.8499755859375
Procentowa jakość regresji: 99.70%

Przykład 3:
Przewidywana wartość: 456.4532470703125
Rzeczywista wartość: 450.54998779296875
Procentowa jakość regresji: 98.69%

Przykład 4:
Przewidywana wartość: 876.0612182617188
Rzeczywista wartość: 839.4600219726562
Procentowa jakość regresji: 95.64%

Przykład 5:
Przewidywana wartość: 697.1907958984375
Rzeczywista wartość: 705.010009765625
Procentowa jakość regresji: 98.89%

Przykład 6:
Przewidywana wartość: 681.84521484375
Rzeczywista wartość: 697.9400024414062
Procentowa jakość regresji: 97.69%

Przykład 7:
Przewidywana wartość: 864.666259765625
Rzeczywista wartość: 854.9099731445312
Procentowa jakość regresji: 98.86%

Przykład 8:
Przewidywana wartość: 794.5511474609375
Rzeczywista wartość: 813.02001953125
Procentowa jakość regresji:

In [66]:
#optimal_model.load_state_dict(torch.load("model_big"))

total_accuracy = 0.0
y_true = []
y_pred = []

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
        y_pred.append(prediction)
    
    y_true.append(sample_data.y.item())
    
    # Wyświetlenie wyników
    # print(f"Przykład {i + 1}:")
    # print("Przewidywana wartość:", prediction)
    # print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    # print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)

In [67]:
from sklearn.metrics import r2_score, mean_squared_error

# Załóżmy, że y_true i y_pred to twoje rzeczywiste i przewidywane wartości.

r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mse**(1/2)

print(f"R2 Score = {r2:.4f}")
print(f"MSE Score = {mse:.4f}")
print(f"RMSE Score = {rmse:.4f}")

y_mean = sum(y_true) / len(y_true)
print(f"Jaki procent zmiennej zależnej stanowi błąd RMSE? {rmse / y_mean * 100:.4f}%")

R2 Score = 0.9913
MSE Score = 540.2464
RMSE Score = 23.2432
Jaki procent zmiennej zależnej stanowi błąd RMSE? 3.8192%
